# Explore the Data for Parking Spots for Disabled Persons

## import the libraries

In [2]:
import geopandas as gpd
import geojson
import json
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder
from OSMPythonTools.api import Api

import seaborn as sns
import missingno as msno
%matplotlib inline

In [3]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 200


In [4]:
from IPython.display import display, HTML

# utility function to display tables, views, etc side-by-side
css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))


## Load Data

In [5]:
df_accessible = gpd.read_file("../data/datathon/parking/parking_spaces_for_disabled_persons.geojson").astype({"ANZAHL": "int64"})
df_inhabitants = pd.read_csv("../data/datathon/demographics/demographics_age_of_inhabitants.csv", thousands=" ", decimal=",")
df_lor = gpd.read_file("../data/datathon/lor_friedrichshain_kreuzberg.geojson")

In [6]:
df_accessible.head()

,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry
0,1,Durchgehend,Friedrichshain,10245,Alt - Stralau 32 b,POINT (13.47776 52.49258)
1,1,Mo-Fr 7-20.30h,Friedrichshain,10245,Alt - Stralau 61,POINT (13.46786 52.49573)
2,1,Mo-Fr 7-17h,Kreuzberg,10969,Alte Jakobstraße 149,POINT (13.39625 52.49921)
3,1,Mo-Fr 10-21h,Kreuzberg,10969,Axel-Springer-Straße 40/41,POINT (13.39887 52.50772)
4,1,Mo-Fr 8-18h,Kreuzberg,10969,Axel-Springer-Straße 44,POINT (13.39943 52.50848)


In [72]:
df_lor.head()

,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,BEZ,FINHALT,STAND,geometry
0,02500831,Wismarplatz,025008,Frankfurter Allee Süd FK,0250,Friedrichshain Ost,02,248991.830677,01.01.2021,"POLYGON ((396010.552 5819339.570, 395991.806 5819343.295, 395974.424 5819346.595, 395963.393 5819348.688, 395949.598 5819351.306, 395939.015 5819353.315, 395925.342 5819355.910, 395903.771 5819360..."
1,02300315,Lausitzer Platz,023003,Nördliche Luisenstadt,0230,Kreuzberg Ost,02,350912.758939,01.01.2021,"POLYGON ((393565.472 5817715.260, 393622.143 5817727.973, 393649.717 5817733.802, 393656.732 5817735.334, 393673.339 5817738.964, 393679.915 5817740.401, 393686.593 5817741.860, 393697.134 5817744..."
2,02500832,Traveplatz,025008,Frankfurter Allee Süd FK,0250,Friedrichshain Ost,02,369572.692540,01.01.2021,"POLYGON ((396230.956 5819298.042, 396219.580 5819255.426, 396213.184 5819231.209, 396206.725 5819206.689, 396200.252 5819182.074, 396195.332 5819163.520, 396189.193 5819139.986, 396180.286 5819118..."
3,02100104,Moritzplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,380552.816915,01.01.2021,"POLYGON ((391710.821 5818713.293, 391709.137 5818711.455, 391693.463 5818694.375, 391682.070 5818683.132, 391656.348 5818657.684, 391633.292 5818635.724, 391611.278 5818615.985, 391584.324 5818591..."
4,02500726,Hausburgviertel,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost,02,309450.362961,01.01.2021,"POLYGON ((394655.527 5820722.533, 394669.046 5820691.408, 394673.664 5820679.888, 394694.711 5820627.390, 394696.040 5820624.073, 394701.121 5820611.161, 394702.501 5820607.654, 394712.189 5820582..."


In [7]:
# set as background map
def get_lor_map():
    bezirk_map = df_lor.explore(
        column="PLR_NAME",
        legend=True,
        legend_kwds=dict(colorbar=False),
        style_kwds=dict(opacity=0.4, fillOpacity=0.7),
        name="Friedrichshain-Kreuzberg"
    )
    return bezirk_map

In [161]:
get_lor_map()

In [9]:
total_accessible_parking_spots = df_accessible.ANZAHL.sum()
total_accessible_parking_spots

157

There are 157 parkink spots for disabled people in Friedrichshain Kreuzberg

now let's look at how many people are in Friedrichshain Kreuzberg

In [26]:
df_inhabitants.head()

,lor_number,total,below age 6,6 - 15,15 - 18,18 - 27,27 - 45,45 - 55,55 - 65,65 an more,of those female,of those immigrants,lor
0,1,6 257,368,508,172,834,2 309,793,629,644,2 912,2 507,Askanischer Platz
1,2,8 633,551,864,313,1 169,2 659,1 077,907,1 093,4 270,3 004,Mehringplatz
2,3,5 385,374,531,155,635,1 732,693,631,634,2 694,1 717,Am Berlin Museum
3,4,5 587,373,538,190,750,1 563,807,719,647,2 744,2 074,Moritzplatz
4,5,9 527,563,940,294,1 144,2 899,1 263,1 092,1 332,4 752,3 290,Prinzenstraße


In [10]:
total_inhabitants = df_inhabitants.total.sum()
total_inhabitants

288906

In [11]:
total_accessible_parking_spots / total_inhabitants * 100

0.05434293507230725

## Number of people with disabilities in Germany according to Statistisches Bundesamt

According to this article [link](https://www.destatis.de/DE/Presse/Pressemitteilungen/2020/06/PD20_230_227.html) there are around 9.8% of the German population living with a disability. This number is for the whole population of Germany and not for Friedrichshain Kreuzberg specifically

But 1 accessible parking spot for every 1840 inhabitants sounds way too few

In [39]:
total_inhabitants / total_accessible_parking_spots


1840.1656050955414

# Show accessible parking spots on the map

In [12]:
df_accessible.explore(
    m=get_lor_map()
)

## get number of accessible parking spots per lor

In [13]:
df_accessible.geometry = df_accessible.geometry.to_crs(25833)


In [14]:
df_accessible.geometry.within(df_lor.geometry)

/Users/lelischiestl/opt/anaconda3/envs/dssg/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

In [46]:
df_test = df_accessible.sjoin(df_lor)

In [16]:
df_test.head()

,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry,index_right,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,BEZ,FINHALT,STAND
0,1,Durchgehend,Friedrichshain,10245,Alt - Stralau 32 b,POINT (396651.507 5816915.423),16,02500836,Stralauer Halbinsel,025008,Frankfurter Allee Süd FK,0250,Friedrichshain Ost,02,1.117020e+06,01.01.2021
1,1,Mo-Fr 7-20.30h,Friedrichshain,10245,Alt - Stralau 61,POINT (395986.790 5817280.079),16,02500836,Stralauer Halbinsel,025008,Frankfurter Allee Süd FK,0250,Friedrichshain Ost,02,1.117020e+06,01.01.2021
2,1,Mo-Fr 7-17h,Kreuzberg,10969,Alte Jakobstraße 149,POINT (391134.205 5817773.345),24,02100103,Am Berlin Museum,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,4.132675e+05,01.01.2021
68,2,Mo-Fr 6-19.30h,Kreuzberg,10969,Oranienstraße 106,POINT (391438.749 5818569.490),24,02100103,Am Berlin Museum,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,4.132675e+05,01.01.2021
3,1,Mo-Fr 10-21h,Kreuzberg,10969,Axel-Springer-Straße 40/41,POINT (391332.977 5818715.085),33,02100102,Mehringplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,6.244703e+05,01.01.2021


In [54]:
df_acc_in_lor = df_test[["index_right", "ANZAHL"]].rename(columns ={"index_right": "lor", "ANZAHL": "count"})
df_acc_per_lor= df_acc_in_lor.groupby("lor").sum()

In [55]:
df_acc_in_lor.sort_values("lor").head()

,lor,count
17,0,1
29,0,2
21,0,2
96,1,1
95,1,1


In [94]:
df_acc_per_lor

,count
lor,
0,5
1,2
3,3
5,4
6,3
7,9
9,6
10,8
12,7


# looking at the time availability of accessible parking spots

In [58]:
df_accessible

,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry
0,1,Durchgehend,Friedrichshain,10245,Alt - Stralau 32 b,POINT (396651.507 5816915.423)
1,1,Mo-Fr 7-20.30h,Friedrichshain,10245,Alt - Stralau 61,POINT (395986.790 5817280.079)
2,1,Mo-Fr 7-17h,Kreuzberg,10969,Alte Jakobstraße 149,POINT (391134.205 5817773.345)
3,1,Mo-Fr 10-21h,Kreuzberg,10969,Axel-Springer-Straße 40/41,POINT (391332.977 5818715.085)
4,1,Mo-Fr 8-18h,Kreuzberg,10969,Axel-Springer-Straße 44,POINT (391373.243 5818799.294)
5,1,Mo-Fr 8-17h,Friedrichshain,10247,Bänschstraße 74,POINT (396096.563 5819697.624)
6,1,Mo-Fr 8-16h,Friedrichshain,10249,Barnimstraße 18,POINT (392979.081 5820731.578)
7,1,Durchgehend,Friedrichshain,10249,Barnimstraße 18,POINT (392988.035 5820722.338)
8,1,Durchgehend,Kreuzberg,10961,Bergmannstraße 25,POINT (391009.718 5816646.440)
9,1,Mo-Fr 8-18h,Kreuzberg,10963,Bernburger Straße 30-31,POINT (390030.176 5818438.097)


In [59]:
df_accessible.BEMERKUNG.unique()

array(['Durchgehend', 'Mo-Fr 7-20.30h', 'Mo-Fr 7-17h', 'Mo-Fr 10-21h',
       'Mo-Fr 8-18h', 'Mo-Fr 8-17h', 'Mo-Fr 8-16h', 'Mo-Fr 8-19h',
       'Sa 8-16h, So 9-18h', '(täglich) 9-22h', 'Mo-Fr 12-18h',
       'Mo-Fr  7-16h', 'Mo - Fr 9-18h, Sa 9-14h', 'Mo-Do 7-19h, Fr 7-16h',
       'Do-So 15-22h', 'Mo-Fr 9-16h', 'Mo-Fr 7-16h',
       'Mo-Fr 9-18h, Sa 9-14h', 'Mo-Fr 8-20h', 'Mo-Fr  6-18h',
       'Mo-Fr 9-21h', 'Mo-Fr  8-18h', 'Mo-Fr 9-18h', 'Mo-Fr 8.30-18h',
       '(täglich) 19-23h', 'Mo-Fr 7-20h', 'werktags 7-18h',
       'Mo-Fr 13-19h', 'Mo-Do 8-18h, Fr 8-13h', 'Mo-Fr 7-18h',
       'Mo-Fr 8-18h, Sa 9-16h', 'Mo Fr 8-16h', 'Mo-Fr 6-18h',
       '(täglich) 9-20h', 'Mo-Do 8-17h, Fr 8-15h', 'Mo-Fr 6.30-20h',
       'Mo-Fr 9-18h, Sa 9-13h', 'Mo-Fr 7-19h', 'Mo-Fr 9-20h',
       'Mo-Fr 6-19.30h', 'Mo-Fr 7-15h', 'Mo-Fr 10-20h',
       'Mo-Fr 6-8h, 13-15h', 'Mo-So 9-21h', 'Mo-Fr 7.30-18h',
       'Mo-Fr 8-19h, Sa 8-16h', 'Sonn- u. Feiertags 10-13h',
       'Mo-Fr 8.30-18h, Sa 9-11h', 'Mo-Fr

In [61]:
df_accessible[["ANZAHL", "BEMERKUNG"]].groupby("BEMERKUNG").count()

,ANZAHL
BEMERKUNG,
(täglich) 18-23h,1
(täglich) 19-23h,1
(täglich) 9-20h,1
(täglich) 9-22h,1
(täglich) 9-23h,1
Do-So 15-22h,1
Durchgehend,24
"Mi 18-21h, So 9-12h und 18-21h",1
"Mo - Fr 9-18h, Sa 9-14h",1


In [71]:
df_accessible[df_accessible.BEMERKUNG.str.match(r"^Mo-Fr")]

,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry
1,1,Mo-Fr 7-20.30h,Friedrichshain,10245,Alt - Stralau 61,POINT (395986.790 5817280.079)
2,1,Mo-Fr 7-17h,Kreuzberg,10969,Alte Jakobstraße 149,POINT (391134.205 5817773.345)
3,1,Mo-Fr 10-21h,Kreuzberg,10969,Axel-Springer-Straße 40/41,POINT (391332.977 5818715.085)
4,1,Mo-Fr 8-18h,Kreuzberg,10969,Axel-Springer-Straße 44,POINT (391373.243 5818799.294)
5,1,Mo-Fr 8-17h,Friedrichshain,10247,Bänschstraße 74,POINT (396096.563 5819697.624)
6,1,Mo-Fr 8-16h,Friedrichshain,10249,Barnimstraße 18,POINT (392979.081 5820731.578)
9,1,Mo-Fr 8-18h,Kreuzberg,10963,Bernburger Straße 30-31,POINT (390030.176 5818438.097)
12,1,Mo-Fr 8-19h,Kreuzberg,10961,Blücherstraße 39,POINT (391706.305 5816803.777)
13,2,Mo-Fr 8-18h,Kreuzberg,10961,Blücherstraße 62,POINT (391266.413 5817308.641)
18,1,Mo-Fr 7-17h,Kreuzberg,10969,Charlottenstraße 13,POINT (390895.919 5818641.217)


# Gif the map

In [72]:
df_accessible.explore()

In [80]:
df_accessible[df_accessible.BEMERKUNG == "Durchgehend"].explore(m=get_lor_map())

In [79]:
fig = df_accessible.explore(m=get_lor_map())
fig

In [87]:
df_accessible.explore(
    legend=True,
    name="Durchgehend",
    legend_kwds={"caption":"test"}
)

In [92]:
def test_map():
    tmap = df_accessible.explore(
        column="ANZAHL",
        legend=True,
        legend_kwds=dict(caption="TEST)-}{(}uaenten", colorbar=False)
    )
    return tmap

In [93]:
test_map()

In [95]:
import folium

In [96]:
m2=folium.Map(location=loc)
folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)

m2

NameError: name 'loc' is not defined

In [151]:


import folium

mapa = folium.Map([52.51, 13.45],
                  zoom_start=13.3,
                  tiles='cartodbpositron')

points = folium.features.GeoJson(df_accessible.geometry)

mapa.add_children(points)

folium.TileLayer()
mapa



/var/folders/0n/_k8_jq_j53g_52n46cdsgp540000gn/T/ipykernel_96511/1827669630.py:9: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  mapa.add_children(points)


import folium
m2=folium.Map(location=loc)
folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)

m2


In [ ]:
m2 = folium.Map([52.51, 13.45],
                  zoom_start=13.3,
                  tiles='cartodbpositron')
folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)

m2

In [ ]:
points = folium.features.GeoJson(df_accessible.geometry)

m2.add_child(points)

In [162]:
m4 = df_accessible.explore()

In [180]:
m7 = folium.Map(
    location=[52.51, 13.45],
    zoom_start=13.3,
    tiles="openstreetmap"
)

points = folium.features.GeoJson(
    df_accessible.geometry,
    marker=folium.Circle()
)
points.add_to(m7)

folium.features.GeoJson(
    df_test.geometry,
    marker=folium.CircleMarker()
).add_to(m7)
folium.LayerControl().add_to(m7)

# folium.features.GeoJson(df_accessible.geometry)

m7


In [171]:
m7

In [181]:
df_with_timestamp = df_accessible

In [183]:
import datetime

In [187]:
df_with_timestamp["time"] = pd.to_datetime("2021-01-01")

In [192]:
df2_with_timestamp = df_accessible[df_accessible.BEMERKUNG == "Durchgehend"]
df2_with_timestamp["time"] = pd.to_datetime("2021-02-01")
df2_with_timestamp


/Users/lelischiestl/opt/anaconda3/envs/dssg/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry,time
0,1,Durchgehend,Friedrichshain,10245,Alt - Stralau 32 b,POINT (396651.507 5816915.423),2021-02-01
7,1,Durchgehend,Friedrichshain,10249,Barnimstraße 18,POINT (392988.035 5820722.338),2021-02-01
8,1,Durchgehend,Kreuzberg,10961,Bergmannstraße 25,POINT (391009.718 5816646.440),2021-02-01
10,1,Durchgehend,Kreuzberg,10961,Blücherplatz 1,POINT (390868.257 5817540.812),2021-02-01
11,3,Durchgehend,Kreuzberg,10961,Blücherplatz 3,POINT (390801.135 5817540.902),2021-02-01
14,1,Durchgehend,Kreuzberg,10967,Böckhstraße 16,POINT (392678.740 5817112.293),2021-02-01
15,1,Durchgehend,Kreuzberg,10967,Böckhstraße Höhe 43 / Ecke Graefestraße 83,POINT (392642.847 5817139.390),2021-02-01
19,1,Durchgehend,Friedrichshain,10243,Erich-Steinfurt-Straße (Ostbahnhof),POINT (393738.623 5819076.499),2021-02-01
33,1,Durchgehend,Kreuzberg,10997,Görlitzer Park,POINT (393843.873 5817566.670),2021-02-01
42,1,Durchgehend,Kreuzberg,10965,Hagelberger Straße 34,POINT (390067.298 5816952.026),2021-02-01


In [195]:
df_with_timestamp = pd.concat([df_with_timestamp, df2_with_timestamp])

In [202]:
from folium.plugins import TimestampedGeoJson

In [211]:
m8 = folium.Map(
    location=[52.51, 13.45],
    zoom_start=13.3,
    tiles="openstreetmap"
)

start_geojson = [
    {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [52.5, 13.5]
        },
        "properties": {"time": "2021-01-01", "style": {"color": ""}, "icon": "circle", "iconstyle": {"fillColor": "#586065", "fillOpacity": 0.8, "stroke": "true", "radius": 5}}},
    {"type": "Feature", "geometry": {"type": "Point", "coordinates": [52.6, 13.5]}, "properties": {"time": "2021-02-01"}},
]
TimestampedGeoJson(start_geojson,period = 'PT1H',
                  duration = 'P1M',
                  transition_time = 1000,
                  auto_play = True).add_to(m8)
m8

In [224]:
df_accessible.head()

,ANZAHL,BEMERKUNG,ORTSTEIL,PLZ,STANDORT,geometry,time
0,1,Durchgehend,Friedrichshain,10245,Alt - Stralau 32 b,POINT (396651.507 5816915.423),2021-01-01
1,1,Mo-Fr 7-20.30h,Friedrichshain,10245,Alt - Stralau 61,POINT (395986.790 5817280.079),2021-01-01
2,1,Mo-Fr 7-17h,Kreuzberg,10969,Alte Jakobstraße 149,POINT (391134.205 5817773.345),2021-01-01
3,1,Mo-Fr 10-21h,Kreuzberg,10969,Axel-Springer-Straße 40/41,POINT (391332.977 5818715.085),2021-01-01
4,1,Mo-Fr 8-18h,Kreuzberg,10969,Axel-Springer-Straße 44,POINT (391373.243 5818799.294),2021-01-01


In [230]:
berlin_map = folium.Map(
    location=[52.51, 13.45],
    zoom_start=13.3,
    tiles="openstreetmap"
)

parking_spot_1 = {
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            52, 13
        ] 
    },
    "properties": {
        "style": {
            "color": "red"
        },
        "times": ['2015-07-22T00:00:00', '2015-08-22T00:00:00',
                  '2015-09-22T00:00:00', '2015-10-22T00:00:00',
                  '2015-11-22T00:00:00', '2015-12-22T00:00:00',
                  '2016-11-22T00:00:00', '2020-11-22T00:00:00']
    }
}

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': [parking_spot_1]},
    period='P1M',
    duration='P1M',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(berlin_map)

berlin_map

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[52.467697, -2.548828], zoom_start=6)

polygon_1 = {
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPolygon',
        'coordinates': [((
             (-2.548828, 51.467697),
             (-0.087891, 51.536086),
             (-1.516113, 53.800651),
             (-6.240234, 53.383328),
        ),)] * 6, # duplication for matching 6 timestamps
    },
    'properties': {
        'style': {
            'color': 'blue',
        },
        'times': ['2015-07-22T00:00:00', '2015-08-22T00:00:00',
                  '2015-09-22T00:00:00', '2015-10-22T00:00:00',
                  '2015-11-22T00:00:00', '2015-12-22T00:00:00']
    }
}

polygon_2 = {
    'type': 'Feature',
    'geometry': {
        'type': 'MultiPolygon',
        'coordinates': [((
             (-3.548828, 50.467697),
             (-1.087891, 50.536086),
             (-2.516113, 52.800651),
             (-7.240234, 52.383328),
        ),)] * 2, # duplication for matching 2 timestamps
    },
    'properties': {
        'style': {
            'color': 'yellow',
        },
        'times': ['2015-07-22T00:00:00', '2015-08-22T00:00:00']
    }
}

TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': [polygon_1, polygon_2]},
    period='P1M',
    duration='P1M',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM/DD',
).add_to(m)
    
m
